In [ ]:
# AUTO click
from pynput.mouse import Controller,Button
import time

mouse = Controller()

while True:
    mouse.click(Button.left,1)
    print('clicked')
    time.sleep(10)

In [ ]:
import cv2,dlib, random,os, csv, time
import numpy as np
from wear_mask.FacialMaskDataSet import FacialMaskDataSet
from wear_mask import MTCNN
from tool.FileFunction import FileFunction
# jupyter notebook   --NotebookApp.allow_origin='https://colab.research.google.com'   --port=8888   --NotebookApp.port_retries=0

# Demo

## Demo wear mask camera

In [ ]:
facial_mask_dataset = FacialMaskDataSet()
file_function =  FileFunction()
#----initialization of MTCNN model
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(os.path.join(os.getcwd(),'./models/shape_predictor_68_face_landmarks.dat'))
# Read mask pictures
maskImagePaths = file_function.getPath(os.path.join(os.getcwd(),"mask_image"))
numberOfMask = len(maskImagePaths)
if numberOfMask <=0:
    print("there are no mask in ")
offsetRandomMask = random.randint(0, numberOfMask - 1)
# Open video
cap = cv2.VideoCapture(0)
ret, frame = cap.read()
timer = time.time()
while(True):
            if (time.time() - timer) >= 2:
                print (time.time()-timer)
                timer = time.time()
                offsetRandomMask = random.randint(0, numberOfMask - 1)
            # capture frame by frame
            ret, frame = cap.read()
            xMin, xMax, yMin, yMax, size, landmarks =facial_mask_dataset.findMouthCoordinate(frame,detector,predictor)
            if size is not None:
                # If can find all land mark use Homography to overlay mask
                landmarks = np.array([[point.x,point.y] for point in landmarks.parts()])
                if (len(landmarks) != 0) and (landmarks is not None) and (landmarks > 0).all():
                    pathRandomMask = maskImagePaths[offsetRandomMask]
                    imageRandomMask = cv2.imread(pathRandomMask, cv2.IMREAD_UNCHANGED)
                    coordinatePointsFace = [[point[0], point[1]] for point in landmarks[1:16:1]]
                    coordinatePointsFace.append([landmarks[29][0],landmarks[29][1]])
                    nameMask = pathRandomMask.split("\\")[-1].split(".")[0]
                    pathAnnotation = os.path.join(os.path.join(os.getcwd(), "mask_annotation"),nameMask+".csv")
                    with open(pathAnnotation) as annotationFile:
                        csvReader = csv.reader(annotationFile)
                        coordinatePointsMask = []
                        for row in csvReader:
                            try:
                                coordinatePointsMask.append([float(row[1]), float(row[2])])
                            except ValueError:
                                continue
                    coordinatePointsFace = np.array(coordinatePointsFace, dtype = np.float)
                    coordinatePointsMask = np.array(coordinatePointsMask, dtype=np.float)
                    frame = facial_mask_dataset.attachMaskToFaceHomoGraphy(frame, imageRandomMask, coordinatePointsMask, coordinatePointsFace )
            cv2.imshow("frame", frame)
            if(cv2.waitKey(1) == ord("q")):
                break
cap.release()
cv2.destroyAllWindows()

# Face recognize

In [ ]:
from train.FaceNet import FaceNetModel,call_instance_FaceNet_with_custom, call_instance_FaceNet_without_custom, call_instance_FaceNet_with_last_isDense,convert_train_model_to_embedding
from PIL import Image
import numpy as np
import tensorflow as tf
from train.Classify import Classify
from tool.FormatFunction import FormatFunction
from tool.FileFunction import FileFunction
from tool.GlobalValue import GlobalValue



global_value = GlobalValue(image_size=[110,110], batch_size = 512, shuffle_size = 1000, ratio_train = 0.8, ratio_test = 0.1, ratio_valid = 0.1, epochs = 40, small_epochs = 50,
                           image_each_class = 15)
format_function = FormatFunction(global_value)
model_path = os.path.join(os.getcwd(),"")
reload_model  = call_instance_FaceNet_with_last_isDense(global_value.IMAGE_SIZE,10575)
reload_model.load_weights(model_path)
embedding_model = convert_train_model_to_embedding(reload_model)


In [ ]:
import numpy as np
vector = [1,2,3]
vector = vector/np.linalg.norm(vector)
print(vector)

print(np.linalg.norm(vector))

In [ ]:
import torch

print(torch.cuda.get_device_name(0))




In [10]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer, Input,  MaxPooling2D, Flatten, Dense,\
    						Dropout, BatchNormalization, Conv2D
from keras import regularizers

import tensorflow as tf


class ArcFace(Layer):
    def __init__(self, n_classes=10, s=30.0, m=0.50, regularizer=None, **kwargs):
        super(ArcFace, self).__init__(**kwargs)
        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.regularizer = regularizers.get(regularizer)

    def build(self, input_shape):
        temp = input_shape[0][1:3]
        super(ArcFace, self).build(input_shape[0])
        temp = input_shape[0][-1]
        self.W = self.add_weight(name='W',
                                 shape=(self.n_classes, input_shape[0][-1]),
                                 initializer='glorot_uniform',
                                 trainable=True,
                                 regularizer=self.regularizer)

    @tf.function
    def call(self, inputs):
        x, y = inputs
        c = K.shape(x)[-1]
        # normalize feature
        x = tf.nn.l2_normalize(x, axis=1)
        # normalize weights
        W = tf.nn.l2_normalize(self.W, axis=0)
        # dot product
        logits = x @ W
        # add margin
        # clip logits to prevent zero division when backward
        theta = tf.acos(K.clip(logits, -1.0 + K.epsilon(), 1.0 - K.epsilon()))
        target_logits = tf.cos(theta + self.m)
        # sin = tf.sqrt(1 - logits**2)
        # cos_m = tf.cos(logits)
        # sin_m = tf.sin(logits)
        # target_logits = logits * cos_m - sin * sin_m
        #
        logits = logits * (1 - y) + target_logits * y
        # feature re-scale
        logits *= self.s
        out = tf.nn.softmax(logits)

        return out

    def compute_output_shape(self, input_shape):
        return (None, self.n_classes)

tf.config.run_functions_eagerly(True)
input = Input(shape=(28, 28, 1))
label = Input(shape=(10,))

x = Conv2D(32, kernel_size=(3, 3), activation='relu')(input)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64, kernel_size=(3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Flatten()(x)
x = Dense(512, kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
output = ArcFace(n_classes=10)([x, label])

model = tf.keras.Model([input, label], output)

# model.compile(loss='categorical_crossentropy',
#               optimizer=Adam(),
#               metrics=['accuracy'])

# model.fit([x_train, y_train],
#           y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_data=([x_test, y_test], y_test),
#           callbacks=[ModelCheckpoint('model.hdf5',
#                      verbose=1, save_best_only=True)])


ValueError: Exception encountered when calling layer "arc_face_9" (type ArcFace).

Dimensions must be equal, but are 512 and 10 for '{{node arc_face_9/matmul}} = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false](arc_face_9/l2_normalize, arc_face_9/l2_normalize_1)' with input shapes: [?,512], [10,512].

Call arguments received by layer "arc_face_9" (type ArcFace):
  • inputs=['tf.Tensor(shape=(None, 512), dtype=float32)', 'tf.Tensor(shape=(None, 10), dtype=float32)']